In [1]:
from langchain.document_loaders import UnstructuredPDFLoader, OnlinePDFLoader
from langchain.text_splitter import RecursiveCharacterTextSplitter

In [37]:
loader = UnstructuredPDFLoader('./qb.pdf')

In [3]:
data = loader.load()

In [4]:
print(f'{len(data[0].page_content)}')

865694


In [5]:
text_splitter=RecursiveCharacterTextSplitter(chunk_size=1000, chunk_overlap=0)
texts = text_splitter.split_documents(data)

In [6]:
print(len(texts))

1087


# Embeddings Portion

In [7]:
from langchain.vectorstores import Pinecone
from langchain.embeddings.openai import OpenAIEmbeddings
import pinecone

/home/sandman/.local/lib/python3.10/site-packages/pinecone/index.py:4: TqdmExperimentalWarning: Using `tqdm.autonotebook.tqdm` in notebook mode. Use `tqdm.tqdm` instead to force console mode (e.g. in jupyter console)
  from tqdm.autonotebook import tqdm


In [8]:
OPENAI_API_KEY = %env OPENAI_API_KEY
embeddings = OpenAIEmbeddings(openai_api_key=OPENAI_API_KEY)

In [12]:
PINECONE_API_KEY = %env PINECONE_API_KEY
pinecone.init(
    api_key=PINECONE_API_KEY,
    environment='northamerica-northeast1-gcp'
)
index_name = "langchain1"

In [15]:
docsearch = Pinecone.from_texts([t.page_content for t in texts], embeddings, index_name=index_name)

In [30]:
query = "What is a qubit in Quantum Mechanics?"
docs = docsearch.similarity_search(query)

In [39]:
len(docs)
print(docs[0].page_content)

with real qubits is extremely challenging and in its infancy, but it has already established that there is no objection in principle to realising the simple unitary operators that quantum computing requires. It is too early to tell what physical form qubits will take when quantum computing becomes a mature technology. Consequently, we leave to one side the question of how our operators are to be realised and focus instead on what operators we require and what could be achieved with them when they have been realised.


# Query Answer

In [32]:
from langchain.llms import OpenAI
from langchain.chains.question_answering import load_qa_chain

In [33]:
llm = OpenAI(model_name="text-ada-001", n=2, best_of=2,openai_api_key=OPENAI_API_KEY)
chain = load_qa_chain(llm, chain_type="stuff")

In [34]:
query = "What is a qubit?"
docs = docsearch.similarity_search(query)

In [36]:
chain.run(input_documents=docs, question=query)

' A quantum bit is a unitary operator that requires two states to be changed, controlled-U operators.'